# Merge data #
For the SentirEs project this code is gathering the ratings six sensory modalities from the partipants:
Audition, Gustatory, Haptic, Interoception, Olfaction, and Vision. The main purpose of this code is to read a folder of csv files
(all the partipants resonses) and concatonate it toda to form one long csv file we can use in our data analysis section. 


In [3]:
import pandas as pd
import numpy as np
import glob
import os

In [4]:
def combine_csv_from_folder(folder_path, columns, output_file, manzanita_arg):
    file_paths = glob.glob(f"{folder_path}/*.csv")
    dataframes = []
    
    # Loop over each CSV file in the folder
    for file_path in file_paths:
        # Read only the required columns from the file
        df = pd.read_csv(file_path, usecols=columns)
        
        # Handling manzanita_arg columns
        df_io = pd.read_csv(file_path, usecols=manzanita_arg)
        
        # Process manzanita_arg columns to add check columns in the main dataframe
        for manzana in manzanita_arg:
            df_manzana = df_io[[manzana]].fillna(pd.NA).dropna()
            
            # Add check column based on the presence of non-NaN values in the `manzana` column
            if len(df_manzana) > 0:
                df[manzana] = 1  # Mark as 1 if data exists
            else:
                df[manzana] = 0  # Mark as 0 if no data
                
        # Drop rows where 'Word' column is NaN
        df.dropna(subset=["Word"], inplace=True)
        (n_items,_)= df.shape
        df["Index"] = list(range(n_items))
        # Append the cleaned dataframe to the list
        dataframes.append(df)
    
    # Combine all the dataframes into one
    combined_df = pd.concat(dataframes, ignore_index=True)
    
    # Apply mask where 'nose2PAN.numClicks' is 0
    mask = combined_df["nose2PAN.numClicks"] == 0
    
    # Replace NaN values in the dataframe with "NaN" as a string
    combined_df.fillna(value="NaN", inplace=True)
    
    # Save the combined dataframe to the specified output CSV file
    combined_df.to_csv(output_file, index=False)

# Example usage:
folder_path = "/Users/soniasimon/Documents/thesis/selecting_files/data"
columns = ["PROLIFIC_PID", "Word", "auditionslider.response", "tasteslider.response", 
           "hapticslider.response", "olfactionslider.response", "visionslider.response", 
           "interoceptionsslider.response", "Category", "participant", "nose2PAN.numClicks",]
manzanita_arg = ["Ave.numClicks", "quinze.numClicks", "manzana.numClicks"]
output_file = "/Users/soniasimon/Documents/thesis/selecting_files/output_4.csv"

combine_csv_from_folder(folder_path, columns, output_file, manzanita_arg)
